# Setting up

Throughout this talk, we will play with the _world_ database used in the documentation of [doobie](https://tpolecat.github.io/doobie), a most convenient JDBC wrapper for Scala. You can find instructions [here](https://tpolecat.github.io/doobie/docs/01-Introduction.html) for the set-up of a postgres database server and the installation of the _world_ database. The script [common.sc](./common.sc) contains several utilities as well as a ready-to-use doobie transactor for issuing SQL queries to the world database.

In [1]:
import $file.common
import common._
import doobie._, doobie.implicits._

Compiling /Users/jserrano/Documents/tagless-final-tutorial/common.sc

import $file.$     

import common._

import doobie._, doobie.implicits._

Test the server!

In [2]:
sql"select 1".query[Int].unique.transact(xa).unsafeRunSync

res1: Int = 1

# Variation  1. Plain and direct SQL 

Let's say that we need to query the database to know the names of the capitals with a population larger than eight million people, together with their country names. The most direct way to bring this data into memory is to write an SQL query: 

In [3]:
val sqlLargeCapitals: ConnectionIO[List[(String, String)]] = 
    sql"""
        | select C.name, X.name 
        | from city as C, country as X 
        | where C.id = X.capital and C.population > 8000000""".stripMargin
        .query[(String, String)]
        .to[List]

sqlLargeCapitals: ConnectionIO[List[(String, String)]] = Suspend(
  BracketCase(
    Suspend(
      PrepareStatement(
        """
 select C.name, X.name 
 from city as C, country as X 
 where C.id = X.capital and C.population > 8000000"""
      )
    ),
    doobie.hi.connection$$$Lambda$3240/630269908@5fde2ec3,
    cats.effect.Bracket$$Lambda$3242/334729804@786dab72
  )
)

A value of type `ConnectionIO[_]` represents a JDBC _program_, a pure value. In order to execute it, this program is compiled into an input/output monad, e.g. `cats.effect.IO`, which is also a pure program, although at a lower-level of abstraction. Finally, the interpreter `unsafeRunSync` of the IO program launches the rockets and the database is accessed!

In [4]:
sqlLargeCapitals
    .transact(xa)            // compiles ConnectionIO[List[(...)]] into IO[List[(String, String)]]
    .unsafeRunSync           // interprets IO[List[...]] into List[(String, String)]
    .timed(1)              // measure average execution time in milliseconds
    .millis

36 millis


res3: List[(String, String)] = List(
  ("Jakarta", "Indonesia"),
  ("Seoul", "South Korea"),
  ("Ciudad de M\u00e9xico", "Mexico"),
  ("Moscow", "Russian Federation")
)

This approach performs quite well according to several quality requirements:
* __Efficiency__. Our data is stored in a SQL database, so this is the most efficient way to retrieve the desired information. 
* __Maintanability__. Lots of people out there know SQL. 

However, all that glitter is not gold. This approach falls short in several respects:
* __Safety__. First, we all know about SQL injection attacks ...
* __Evolvability__. We incur in a tight coupling to relational databases. What if we eventually want to change our persistence layer?
* __Modularity__. It's not possible to decompose our queries into smaller pieces that can be reused, tested and composed, so that we can effectively build large query libraries. Much the same way in which we program with functions, classes, etc., in general-purpose languages. Let's dive into this issue in the next [notebook](Variation2.InMemory.ipynb). 